In [179]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Import data

In [174]:
#bids = pd.read_csv('data/bids.csv', sep=',')
#print('bids: OK {}'.format(bids.shape))
#bids.head()
bids = pd.read_csv('data/bids_normalized.csv', sep=',')
print('bids: OK {}'.format(bids.shape))
bids.head()

bids: OK (7656334, 9)


,bid_id,bidder_id,auction,device,country,ip,url,time_normalized,outcome
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,phone0,us,69.166.231.58,vasstdc27m7nks3,2419200.0,NaN
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,phone1,in,50.201.125.84,jmqlhflrzwuay9c,2419200.0,0.0
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,phone2,py,112.54.208.157,vasstdc27m7nks3,2419200.0,NaN
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,phone4,in,18.99.175.133,vasstdc27m7nks3,2419200.0,NaN
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,phone5,in,145.138.5.37,vasstdc27m7nks3,2419200.0,NaN


In [175]:
#b_train = pd.read_csv('data/train.csv', sep=',')
#print('b_train: OK: {}'.format(b_train.shape))
#b_train.head()
b_train = pd.read_csv('data/b_train.csv', sep=',')
print('b_train: OK: {}'.format(b_train.shape))
b_train.head()

b_train: OK: (2013, 10)


,bidder_id,outcome,total_bids,total_auctions,bids_per_auction,mean_time_diff,mean_response,min_response,ip_entropy,url_entropy
0,91a3c57b13234af24875c56fb7e2b2f4rb56a,0.0,24,18,1.3333,10851.13,10673.3750,0.0,49.8149,0.0000
1,624f258b49e77713fc34034560f93fb3hu3jo,0.0,3,1,3.0000,61438.00,2.3333,0.0,0.6931,0.6931
2,1c5f4fc669099bfbfac515cd26997bd12ruaj,0.0,4,4,1.0000,45201.00,219.0000,79.0,1.7918,1.0986
3,4bee9aba2abda51bf43d639013d6efe12iycd,0.0,1,1,1.0000,NaN,330.0000,330.0,0.0000,0.0000
4,4ab12bc61c82ddd9c2d65e60555808acqgos1,0.0,155,23,6.7391,1477.80,35.2258,0.0,601.0291,460.9263


In [176]:
#b_test = pd.read_csv('data/test.csv', sep=',')
#print('b_test: OK: {}'.format(b_test.shape))
#b_test.head()
b_test = pd.read_csv('data/b_test.csv', sep=',')
print('b_test: OK: {}'.format(b_test.shape))
b_test.head()

b_test: OK: (4700, 9)


,bidder_id,total_bids,total_auctions,bids_per_auction,mean_time_diff,mean_response,min_response,ip_entropy,url_entropy
0,49bb5a3c944b8fc337981cc7a9ccae41u31d7,4,3,1.3333,444750.0000,16.7500,0.0,1.7918,1.7918
1,a921612b85a1494456e74c09393ccb65ylp4y,3,2,1.5000,722019.5000,371.3333,2.0,0.6931,0.0000
2,6b601e72a4d264dab9ace9d7b229b47479v6i,17,14,1.2143,345.6250,186.3529,0.0,17.3592,8.8929
3,eaf0ed0afc9689779417274b4791726cn5udi,148,90,1.6444,9890.5510,7762.0946,0.0,580.2510,384.8401
4,cdecd8d02ed8c6037e38042c7745f688mx5sf,23,20,1.1500,5678.2273,9168.6957,16.0,48.4712,0.0000


# PREPROCESSING

# bids_normalized table

In [ ]:
bids.head()

### 1. time_normalized

In [ ]:
min_time = bids['time'].min()
bids['time_normalized'] = round((bids['time'] - min_time) * 19 / 1e9,2)

In [ ]:
pd.DataFrame(bids.time_normalized.unique())
bids.head()

### 2. outcome

In [ ]:
bids['outcome'] = bids.merge(b_train, how='left', on='bidder_id').outcome

### other

In [ ]:
del bids['merchandise']

In [ ]:
del bids['device']
del bids['country']
del bids['ip']
del bids['url']

In [ ]:
bids.to_csv('data/bids_normalized_short.csv', sep=',', header=True, index=False)

# features for b_train & b_test table

In [ ]:
del b_train['address']
del b_train['payment_account']

del b_test['address']
del b_test['payment_account']

### 1. mean_time_diff

In [ ]:
b_train['mean_time_diff'] = 0
for bidder in b_train['bidder_id']:
    #d = bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']] - bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']].shift(1)
    #b_train.loc[b_train[b_train.bidder_id == bidder].index,'mean_time_diff'] = (bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']] - bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']].shift(1)).mean()
    
    m = (bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']] - 
         bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']].shift(1)).mean()[0]
    b_train.loc[b_train[b_train.bidder_id == bidder].index, 'mean_time_diff'] = m
    # b_train.to_csv('data/b_train.csv', sep=',', header=True, index=False)
b_train.loc[:,'mean_time_diff'] = round(b_train.mean_time_diff, 4)

In [ ]:
b_test['mean_time_diff'] = 0
i = 0
for bidder in b_test['bidder_id']:
    if i % 200 == 0:
        print(i)
    i += 1
    m = (bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']] - 
         bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']].shift(1)).mean()[0]
    b_test.loc[b_test[b_test.bidder_id == bidder].index, 'mean_time_diff'] = m
    # b_test.to_csv('data/b_test.csv', sep=',', header=True, index=False)
b_test.loc[:,'mean_time_diff'] = round(b_test.mean_time_diff, 4)

### 2. total_bids

In [ ]:
b_train['total_bids'] = 0
i = 0
for bidder in b_train['bidder_id']:
    if i % 100 == 0:
        print(i)
    i += 1
    c = bids[bids.bidder_id == bidder].count()[0]
    b_train.loc[b_train[b_train.bidder_id == bidder].index, 'total_bids'] = c
    # b_train.to_csv('data/b_train.csv', sep=',', header=True, index=False)

In [ ]:
b_test['total_bids'] = 0
i = 0
for bidder in b_test['bidder_id']:
    if i % 100 == 0:
        print(i)
    i += 1
    c = bids[bids.bidder_id == bidder].count()[0]
    b_test.loc[b_test[b_test.bidder_id == bidder].index, 'total_bids'] = c
    # b_test.to_csv('data/b_test.csv', sep=',', header=True, index=False)

### 3. total_auctions

In [ ]:
b_train['total_auctions'] = 0
i = 0
for bidder in b_train['bidder_id']:
    if i % 100 == 0:
        print(i)
    i += 1
    c = bids[bids.bidder_id == bidder]['auction'].nunique()
    b_train.loc[b_train[b_train.bidder_id == bidder].index, 'total_auctions'] = c
    # b_train.to_csv('data/b_train.csv', sep=',', header=True, index=False)

In [ ]:
b_test['total_auctions'] = 0
i = 0
for bidder in b_test['bidder_id']:
    if i % 100 == 0:
        print(i)
    i += 1
    c = bids[bids.bidder_id == bidder]['auction'].nunique()
    b_test.loc[b_test[b_test.bidder_id == bidder].index, 'total_auctions'] = c
    # b_test.to_csv('data/b_test.csv', sep=',', header=True, index=False)

### 4. bids_per_auction

In [143]:
b_train['bids_per_auction'] = b_train.total_bids.divide(b_train.total_auctions)
b_train['bids_per_auction'] = round(b_train['bids_per_auction'], 4) 

In [144]:
b_test['bids_per_auction'] = b_test.total_bids.divide(b_test.total_auctions)
b_test['bids_per_auction'] = round(b_test['bids_per_auction'], 4)

### 5-6. mean_response & min_response

In [ ]:
# create additional dataset bids_sorted to generate values for feature mean_response & min_response
# already created

bids_sorted = bids.sort_values(by=['auction', 'time'])
del bids_sorted['time']
del bids_sorted['outcome']
bids_sorted['diff'] = bids_sorted.time_normalized - bids_sorted.time_normalized.shift(1)
i = 0
for auc in bids_sorted.auction.unique():
    if i % 200 == 0:
        print(i)
    i += 1
    bids_sorted.loc[bids_sorted[bids_sorted.auction == auc].index, 'diff'] = bids_sorted[bids_sorted.auction == auc].time_normalized - bids_sorted[bids_sorted.auction == auc].time_normalized.shift(1)
    bids_sorted.loc[bids_sorted[bids_sorted.auction == auc].index[0], 'diff'] = np.NaN
bids_sorted.to_csv('data/bids_sorted.csv', sep=',', header=True, index=False)

In [14]:
bids_sorted = pd.read_csv('data/bids_sorted.csv', sep=',')
print('bids_sorted: OK {}'.format(bids_sorted.shape))
bids_sorted.head()

bids_sorted: OK (7656334, 5)


,bid_id,bidder_id,auction,time_normalized,diff
0,5826422,92af1e40713e077ef87f5352fb56772fjnzm7,00270,1275528.0,NaN
1,5834284,a939f51234ad2c44eb9ccc84e754f41foiv7g,00270,1276446.0,918.0
2,5850448,92af1e40713e077ef87f5352fb56772fjnzm7,00270,1278300.0,1854.0
3,5884054,9655ccc7c0c193f1549475f02c54dce45kjw7,00270,1282037.0,3737.0
4,5898318,9655ccc7c0c193f1549475f02c54dce45kjw7,00270,1283595.0,1558.0


In [ ]:
b_train['mean_response'] = 0
i = 0
for bidder in b_train['bidder_id'][i:]:
    if i % 100 == 0:
        print(i)
    i += 1
    temp = bids_sorted[bids_sorted.bidder_id == bidder]['diff']
    b_train.loc[b_train[b_train.bidder_id == bidder].index, ['mean_response', 'min_response']] = \
        round(temp.mean(), 4), round(temp.min(), 4)
    # b_train.to_csv('data/b_train.csv', sep=',', header=True, index=False)

In [ ]:
b_test['mean_response'] = 0
i = 0
for bidder in b_test['bidder_id'][i:]:
    if i % 100 == 0:
        print(i)
    i += 1
    temp = bids_sorted[bids_sorted.bidder_id == bidder]['diff']
    b_test.loc[b_test[b_test.bidder_id == bidder].index, ['mean_response', 'min_response']] = \
        round(temp.mean(), 4), round(temp.min(), 4)
    # b_test.to_csv('data/b_test.csv', sep=',', header=True, index=False)

### 7-8. ip_entropy, url_entropy

In [86]:
def log_entropy(x):
    e = np.sum(np.log(np.array(range(1,np.sum(x)))))
    for i in x:
        e -= np.sum(np.log(np.array(range(1,i))))
    return e

In [ ]:
b_train['ip_entropy'] = 0
b_train['url_entropy'] = 0
i = 0
for bidder in b_train['bidder_id'][i:]:
    if i % 100 == 0:
        print(i)
    i += 1
    temp_df = bids[bids.bidder_id == bidder][['ip','url']]
    b_train.loc[b_train[b_train.bidder_id == bidder].index, ['ip_entropy', 'url_entropy']] = \
                    round(log_entropy(temp_df.ip.groupby(temp_df.ip).count()),4), \
                    round(log_entropy(temp_df.url.groupby(temp_df.url).count()),4)
    b_train.to_csv('data/b_train.csv', sep=',', header=True, index=False)

In [ ]:
b_test['ip_entropy'] = 0
b_test['url_entropy'] = 0
i = 0
for bidder in b_test['bidder_id'][i:]:
    if i % 100 == 0:
        print(i)
    i += 1
    temp_df = bids[bids.bidder_id == bidder][['ip','url']]
    b_test.loc[b_test[b_test.bidder_id == bidder].index, ['ip_entropy', 'url_entropy']] = \
                    round(log_entropy(temp_df.ip.groupby(temp_df.ip).count()),4), \
                    round(log_entropy(temp_df.url.groupby(temp_df.url).count()),4)
    b_test.to_csv('data/b_test.csv', sep=',', header=True, index=False)

In [134]:
df = pd.DataFrame([1,2,1], columns=['h'])
print(log_entropy(df.h.groupby(df.h).count()))
print(df.h.groupby(df.h).count())
df

0.69314718056
h
1    2
2    1
Name: h, dtype: int64


,h
0,1
1,2
2,1


# look through

In [161]:
features = ['bidder_id', 'outcome',
         'total_bids', 'total_auctions', 'bids_per_auction',
         'mean_time_diff', 'mean_response', 'min_response',
         'ip_entropy', 'url_entropy']

In [178]:
b_train.tail()

,bidder_id,outcome,total_bids,total_auctions,bids_per_auction,mean_time_diff,mean_response,min_response,ip_entropy,url_entropy
2008,369515b3af4f8ca582f90271d30b14b6r52aw,0.0,36,25,1.44,39884.09,2289.6571,4.0,26.087,9.8851
2009,f939c17ffc7c39ac9b35b69e5e75179fv9pe2,0.0,1,1,1.00,NaN,4.0000,4.0,0.000,0.0000
2010,c806dbb2decba0ed3c4ff5e2e60a74c2wjvbl,0.0,2,1,2.00,186171.00,0.5000,0.0,0.000,0.0000
2011,0381a69b7a061e9ace2798fd48f1f537mgq57,0.0,1,1,1.00,NaN,0.0000,0.0,0.000,0.0000
2012,84a769adc98498f52debfe57b93a0789556f4,0.0,2,1,2.00,34.00,1918.0000,34.0,0.000,0.0000


In [177]:
b_test.tail()

,bidder_id,total_bids,total_auctions,bids_per_auction,mean_time_diff,mean_response,min_response,ip_entropy,url_entropy
4695,bef56983ba78b2ee064443ae95972877jfkyd,983,41,23.9756,1301.9297,39.2584,0.0,0.0,0.0
4696,4da45cc915c32d4368ac7e773d92d4affwqrr,66,32,2.0625,3913.2000,124.5909,0.0,0.0,0.0
4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,2162,419,5.1599,679.4951,9079.5294,0.0,0.0,0.0
4698,4981c32c54dde65b79dbc48fd9ab6457caqze,5,5,1.0000,43416.5000,35.8000,10.0,0.0,0.0
4699,7ade70030d559a6c255be2f6feca17acnrqs0,382,116,3.2931,3842.1024,161.9215,0.0,0.0,0.0


# save

In [164]:
b_train.to_csv('data/b_train.csv', sep=',', header=True, index=False)

In [165]:
b_test.to_csv('data/b_test.csv', sep=',', header=True, index=False)